In [11]:
import yfinance as yahooFinance

import datetime
 
startDate = datetime.datetime(2006, 1, 1)
endDate = datetime.datetime(2020, 12, 31)

vti = yahooFinance.Ticker("VTI").history(start=startDate,end=endDate)
agg = yahooFinance.Ticker("AGG").history(start=startDate,end=endDate)
dbc = yahooFinance.Ticker("DBC").history(start=startDate,end=endDate)
vix = yahooFinance.Ticker("^VIX").history(start=startDate,end=endDate)
 
all_assets = [vti, agg, dbc, vix]

for asset in all_assets:
    print(asset)
    if max(asset['Dividends'] > 0):
        print("HAS DIVIDENDS")
    if max(asset['Stock Splits'] > 0):
        print("HAS STOCK SPLITS")
    if 'Capital Gains' in asset.columns and max(asset['Capital Gains'] > 0):
        print("Has capital gains")
    # print(asset)
    # print(asset.history(start=startDate,end=endDate))#['Close'])

                                 Open        High         Low       Close  \
Date                                                                        
2006-01-03 00:00:00-05:00   44.173327   44.798203   43.869817   44.687511   
2006-01-04 00:00:00-05:00   44.748223   45.012455   44.723228   45.012455   
2006-01-05 00:00:00-05:00   44.980311   45.030301   44.833911   44.983883   
2006-01-06 00:00:00-05:00   45.287389   45.444499   45.073147   45.426647   
2006-01-09 00:00:00-05:00   45.465921   45.662309   45.426643   45.594467   
...                               ...         ...         ...         ...   
2020-12-23 00:00:00-05:00  184.374826  184.917532  183.908305  184.022552   
2020-12-24 00:00:00-05:00  184.387755  184.502466  183.775933  184.330399   
2020-12-28 00:00:00-05:00  186.022430  186.031999  185.267219  185.496643   
2020-12-29 00:00:00-05:00  186.280558  186.366592  184.330383  184.722321   
2020-12-30 00:00:00-05:00  185.362827  185.821703  185.085601  185.219437   

In [10]:
asset.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')